In [ ]:
# # This Python 3 environment comes with many helpful analytics libraries installed
# # It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# # For example, here's several helpful packages to load

# import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# # Input data files are available in the read-only "../input/" directory
# # For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# # You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# # You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
# ===============================
# Cell 1 :  paths + quick sanity
# ===============================
import os, glob, json, random
from pathlib import Path
import pandas as pd
from collections import Counter
import torch

# -------- 1. Paths --------
ROOT = Path('/kaggle/input/identity-employees-in-surveillance-cctv/dataset')
TRAIN_IMG_DIR = ROOT/'train/images'
TEST_IMG_DIR  = ROOT/'test/images'
LABELS_CSV    = ROOT/'train/labels.csv'

assert TRAIN_IMG_DIR.exists(), f"Missing: {TRAIN_IMG_DIR}"
assert TEST_IMG_DIR.exists(),  f"Missing: {TEST_IMG_DIR}"
assert LABELS_CSV.exists(),    f"Missing: {LABELS_CSV}"

# -------- 2. GPU check (useful later) --------
print("CUDA available :", torch.cuda.is_available())
if torch.cuda.is_available():
    print("GPU name       :", torch.cuda.get_device_name(0))

# -------- 3. Quick stats --------
train_files = glob.glob(str(TRAIN_IMG_DIR/'*.jpg'))
test_files  = glob.glob(str(TEST_IMG_DIR/'*.jpg'))
print(f"Train images   : {len(train_files):,}")
print(f"Test  images   : {len(test_files):,}")

labels_df = pd.read_csv(LABELS_CSV)
display(labels_df.head())

emp_counts = Counter(labels_df['emp_id'])
print(f"Unique employees: {len(emp_counts)}")
print("Five smallest classes:", emp_counts.most_common()[-5:])


CUDA available : True
GPU name       : Tesla T4
Train images   : 1,179
Test  images   : 636


,filename,emp_id
0,face_0568.jpg,emp016
1,face_0433.jpg,emp014
2,face_1751.jpg,emp004
3,face_0675.jpg,emp028
4,face_0112.jpg,emp001


Unique employees: 34
Five smallest classes: [('emp017', 4), ('emp032', 4), ('emp026', 3), ('emp005', 2), ('emp024', 1)]


In [2]:
# ================================================================
# Cell A : extract 500 JPG frames / employee from reference video
#          + save them in EXTRA_DIR  +  create new_rows list
# ================================================================
import cv2, numpy as np
from pathlib import Path

EXTRA_DIR = Path('/kaggle/tmp_ref_frames')
EXTRA_DIR.mkdir(parents=True, exist_ok=True)

new_rows = []                 # ← will be concatenated with labels_df in Cell 2
SAMPLES  = 500                # number of evenly-spaced frames per video

for emp_dir in (ROOT / 'reference_faces').iterdir():        # emp001, emp002 …
    vid_files = list(emp_dir.glob('*.mp4'))
    if not vid_files:
        continue                               # some employees may have only JPGs
    vid = vid_files[0]
    cap = cv2.VideoCapture(str(vid))
    if not cap.isOpened():
        print(f"⚠️ could not open {vid}")
        continue

    tot = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    # ----- sample SAMPLES indices evenly across the video ----------
    for f in np.linspace(0, tot - 1, SAMPLES, dtype=int):
        cap.set(cv2.CAP_PROP_POS_FRAMES, f)
        ok, frame = cap.read()
        if not ok:
            continue
        fname = f'{emp_dir.name}_{f:04d}.jpg'               # emp001_0003.jpg …
        cv2.imwrite(str(EXTRA_DIR / fname), frame)
        new_rows.append({'filename': fname, 'emp_id': emp_dir.name})

    cap.release()

print(f"✅ Saved {len(new_rows):,} extra frames to {EXTRA_DIR}")


✅ Saved 17,000 extra frames to /kaggle/tmp_ref_frames


In [13]:
# ================================================================
# Cell 2 : dataset, transforms, stratified split (CCTV hold-out) - new
# ================================================================
import subprocess, sys, importlib.util, numpy as np, torch
import torchvision.transforms as T
from torch.utils.data import Dataset, DataLoader, Subset
from sklearn.model_selection import StratifiedShuffleSplit
from collections import Counter
from pathlib import Path
from PIL import Image
import pandas as pd
import random
from io import BytesIO

# 0️⃣ Install timm (once)
if importlib.util.find_spec("timm") is None:
    subprocess.run([sys.executable, "-m", "pip", "-q", "install",
                    "timm", "torchmetrics", "--no-progress"])

# 1️⃣ CCTV labels + video‐frame extras (Cell A)
labels_df = pd.read_csv(LABELS_CSV)      # 1,179 CCTV crops
extra_df  = pd.DataFrame(new_rows)       # e.g. 300 frames × each emp
labels_df = pd.concat([labels_df, extra_df], ignore_index=True)

# 1.1️⃣ Add static reference JPGs as “photo” rows
static_rows = []
for emp_dir in (ROOT/'reference_faces').iterdir():
    for jpg in emp_dir.glob('*.jpg'):
        static_rows.append({'filename': jpg.name, 'emp_id': emp_dir.name})
        # Symlink into EXTRA_DIR for gallery lookup
        if not (EXTRA_DIR/jpg.name).exists():
            (EXTRA_DIR/jpg.name).symlink_to(jpg)
labels_df = pd.concat([labels_df, pd.DataFrame(static_rows)], ignore_index=True)

# 2️⃣ Map employee IDs → numeric labels
employee_ids = sorted(labels_df.emp_id.unique())
emp2idx      = {e:i for i,e in enumerate(employee_ids)}
labels_df["label_idx"] = labels_df.emp_id.map(emp2idx)

# 3️⃣ Tag each row as CCTV vs “photo” (frame or static JPG)
labels_df["source"] = labels_df.filename.apply(
    lambda fn: "photo" if (EXTRA_DIR/fn).exists() else "cctv"
)

# 4️⃣ Paths to look for images
IMG_ROOTS = [TRAIN_IMG_DIR, EXTRA_DIR]

# — custom JPEG-compression transform (PIL-only) ———————
class JpegCompression:
    def __init__(self, quality=(30,60)):
        self.qmin, self.qmax = quality
    def __call__(self, img: Image.Image) -> Image.Image:
        buf = BytesIO()
        q   = random.randint(self.qmin, self.qmax)
        img.save(buf, format="JPEG", quality=q)
        buf.seek(0)
        return Image.open(buf)

# 5️⃣ Heavy-augmented train transforms
train_tfms = T.Compose([
    # PIL-based “surveillance” degradations
    T.RandomResizedCrop(224, scale=(0.8,1.0)),
    T.RandomHorizontalFlip(0.5),
    T.ColorJitter(0.2,0.2,0.2,0.1),
    T.RandomRotation(15),
    T.RandomPerspective(distortion_scale=0.3, p=0.2),
    T.RandomApply([T.GaussianBlur(3)], p=0.2),
    T.RandomApply([JpegCompression((30,60))], p=0.3),

    # convert to tensor + normalize
    T.ToTensor(),
    T.Normalize([0.485,0.456,0.406], [0.229,0.224,0.225]),

    # occlusion simulation (must go after ToTensor)
    T.RandomErasing(p=0.3, scale=(0.02,0.10), ratio=(0.3,3.3)),
])

# 6️⃣ Validation transforms (clean)
val_tfms = T.Compose([
    T.Resize(256), T.CenterCrop(224),
    T.ToTensor(),
    T.Normalize([0.485,0.456,0.406], [0.229,0.224,0.225]),
])

# 7️⃣ Dataset class (multi-root lookup)
class FaceDS(Dataset):
    def __init__(self, df, tfm):
        self.df, self.tfm = df.reset_index(drop=True), tfm
    def __len__(self):
        return len(self.df)
    def __getitem__(self, i):
        row = self.df.iloc[i]
        for root in IMG_ROOTS:
            path = root/row.filename
            if path.exists():
                img = Image.open(path).convert("RGB")
                break
        else:
            raise FileNotFoundError(f"Missing {row.filename}")
        return self.tfm(img), row.label_idx

full_ds = FaceDS(labels_df, train_tfms)

# 8️⃣ Stratified 10 % hold-out on CCTV only (photos always train)
cctv_idx  = labels_df.index[labels_df.source=="cctv"].to_numpy()
cctv_lbls = labels_df.label_idx.loc[cctv_idx].to_numpy()

# keep singleton classes entirely in train
from collections import Counter
cnt = Counter(cctv_lbls)
single_cls = [cls for cls,c in cnt.items() if c==1]
mask_single = np.isin(cctv_lbls, single_cls)
single_idx  = cctv_idx[mask_single]
multi_idx   = cctv_idx[~mask_single]

from sklearn.model_selection import StratifiedShuffleSplit
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.10, random_state=42)
train_multi, val_multi = next(sss.split(
    multi_idx,
    labels_df.label_idx.loc[multi_idx]
))
train_cctv = np.concatenate([single_idx, multi_idx[train_multi]])
val_cctv   = multi_idx[val_multi]

photo_idx = labels_df.index[labels_df.source=="photo"].to_numpy()

train_idx = np.concatenate([train_cctv, photo_idx])
val_idx   = val_cctv

# 9️⃣ DataLoaders
train_ds = Subset(full_ds, train_idx)
val_ds   = Subset(FaceDS(labels_df.loc[val_idx], val_tfms),
                  np.arange(len(val_idx)))

BATCH = 32
train_dl = DataLoader(train_ds, batch_size=BATCH, shuffle=True,
                      num_workers=4, pin_memory=True)
val_dl   = DataLoader(val_ds,   batch_size=BATCH, shuffle=False,
                      num_workers=4, pin_memory=True)

print(f"Train images: {len(train_ds):,} | Val images: {len(val_ds):,}")
print("Smallest training classes:",
      Counter(labels_df.label_idx.loc[train_idx]).most_common()[-5:])


Train images: 18,513 | Val images: 118
Smallest training classes: [(17, 517), (5, 515), (29, 512), (24, 511), (0, 15)]


In [14]:
# # ───────────────────────────────────────────────────────────────
# # Cell 2.5 : Quick A/B Macro-Accuracy check (3 epochs only)
# # ───────────────────────────────────────────────────────────────
# import numpy as np
# import timm, torch
# from sklearn.model_selection import StratifiedShuffleSplit
# from torch.utils.data import Subset, DataLoader
# from torchmetrics.classification import MulticlassAccuracy

# device = 'cuda' if torch.cuda.is_available() else 'cpu'

# def run_3ep(df, label):
#     # build new train/val indices just like in Cell 2 …
#     cctv_idx  = df.index[df.source == "cctv"].to_numpy()
#     cctv_lbls = df.label_idx.loc[cctv_idx].to_numpy()
#     from collections import Counter
#     cnt = Counter(cctv_lbls)
#     single = {cls for cls,c in cnt.items() if c == 1}
#     mask_s  = np.isin(cctv_lbls, list(single))
#     single_i= cctv_idx[mask_s]
#     multi_i = cctv_idx[~mask_s]
#     sss     = StratifiedShuffleSplit(1, test_size=0.10, random_state=0)
#     tr_m, va_m = next(sss.split(multi_i, df.label_idx.loc[multi_i]))
#     train_i = np.concatenate([single_i, multi_i[tr_m], 
#                               df.index[df.source=="photo"].to_numpy()])
#     val_i   = multi_i[va_m]

#     # build DataLoaders
#     tr_ds = Subset(FaceDS(df.loc[train_i], train_tfms),
#                    np.arange(len(train_i)))
#     vl_ds = Subset(FaceDS(df.loc[val_i],   val_tfms),
#                    np.arange(len(val_i)))
#     tr_dl = DataLoader(tr_ds, batch_size=64, shuffle=True,  num_workers=2)
#     vl_dl = DataLoader(vl_ds, batch_size=64, shuffle=False, num_workers=2)

#     # lightweight 3-epoch run
#     model = timm.create_model('mobilenetv2_100',
#                               pretrained=True,
#                               num_classes=len(employee_ids)).to(device)
#     opt   = torch.optim.AdamW(model.parameters(), lr=3e-4)
#     metr  = MulticlassAccuracy(num_classes=len(employee_ids),
#                                average='macro').to(device)

#     print(f"\n>>> {label} | train {len(tr_ds):,} → val {len(vl_ds):,}")
#     for ep in range(1,4):
#         model.train()
#         for x,y in tr_dl:
#             x,y = x.to(device), y.to(device)
#             opt.zero_grad()
#             loss = torch.nn.functional.cross_entropy(model(x), y)
#             loss.backward(); opt.step()
#         model.eval(); metr.reset()
#         with torch.no_grad():
#             for x,y in vl_dl:
#                 x,y = x.to(device), y.to(device)
#                 metr.update(model(x).argmax(1), y)
#         print(f" Epoch {ep} | Val Macro {metr.compute():.4f}")

# # run both variants
# run_3ep(labels_df[labels_df.source=="cctv"], "CCTV only")
# run_3ep(labels_df,                         "CCTV + video/photo")



>>> CCTV only | train 1,061 → val 118
 Epoch 1 | Val Macro 0.1323
 Epoch 2 | Val Macro 0.2026
 Epoch 3 | Val Macro 0.2439

>>> CCTV + video/photo | train 18,513 → val 118
 Epoch 1 | Val Macro 0.2895
 Epoch 2 | Val Macro 0.4789
 Epoch 3 | Val Macro 0.5492


In [16]:
# # --------------------------------------
# # Check for employees with no CCTV crop
# # --------------------------------------
# import pandas as pd

# # Build a DataFrame of counts by (emp_id, source)
# counts = pd.crosstab(labels_df.emp_id, labels_df.source)
# # Employees where cctv==0 & photo>0
# clean_only = counts[(counts.cctv==0) & (counts.photo>0)]
# print("Employees with only 'photo' examples:", clean_only.index.tolist())


Employees with only 'photo' examples: ['emp029']


In [19]:
# ================================================================
# Cell 3 : EfficientNet-B0 + ArcFace fine-tune (100 epochs)
# ================================================================
import timm, torch, math, numpy as np
from torch import nn
from torchmetrics.classification import MulticlassAccuracy

device       = 'cuda' if torch.cuda.is_available() else 'cpu'
NUM_CLASSES  = len(employee_ids)                 # set in Cell 2
EPOCHS       = 50
CKPT         = '/kaggle/working/effb0_arcface.pt'

# ── 1️⃣  Backbone + 512-D projection ────────────────────────────
backbone = timm.create_model(
    'efficientnet_b0', pretrained=True, num_classes=0, global_pool='avg'
).to(device)
proj = nn.Linear(backbone.num_features, 512, bias=False).to(device)

# ── 2️⃣  ArcFace additive-margin head ───────────────────────────
class ArcFace(nn.Module):
    def __init__(self, in_f, out_f, s=30.0, m=0.50):
        super().__init__()
        self.W = nn.Parameter(torch.randn(out_f, in_f))
        nn.init.xavier_uniform_(self.W)
        self.s, self.m = s, m
        self.cos_m, self.sin_m = math.cos(m), math.sin(m)
        self.th, self.mm = math.cos(math.pi - m), math.sin(math.pi - m) * m
    def forward(self, x, labels):
        x_n = nn.functional.normalize(x)
        w_n = nn.functional.normalize(self.W)
        cos = nn.functional.linear(x_n, w_n)            # cosine θ
        sin = torch.sqrt(1.0 - torch.clamp(cos**2, 0, 1))
        phi = cos * self.cos_m - sin * self.sin_m        # cos(θ+m)
        phi = torch.where(cos > self.th, phi, cos - self.mm)
        one_hot = torch.zeros_like(cos, device=x.device)
        one_hot.scatter_(1, labels.view(-1,1), 1.0)
        logits = (one_hot * phi + (1.0 - one_hot) * cos) * self.s
        return logits

arc = ArcFace(512, NUM_CLASSES).to(device)

# ── 3️⃣  Optimizer, warm-up, cosine LR ─────────────────────────
params = list(backbone.parameters()) + list(proj.parameters()) + list(arc.parameters())
opt    = torch.optim.AdamW(params, lr=3e-4, weight_decay=1e-4)
warm   = torch.optim.lr_scheduler.LinearLR(opt, start_factor=0.1, total_iters=2)
sched  = torch.optim.lr_scheduler.CosineAnnealingLR(opt, T_max=EPOCHS)

criterion = nn.CrossEntropyLoss(label_smoothing=0.1)
metric    = MulticlassAccuracy(num_classes=NUM_CLASSES, average='macro').to(device)
best_acc  = 0.0

# ── 4️⃣  Training loop ─────────────────────────────────────────
for ep in range(1, EPOCHS + 1):
    # ---- train ----
    backbone.train(); proj.train(); arc.train()
    for x, y in train_dl:                       # train_dl from Cell 2
        x, y = x.to(device), y.to(device)
        feats  = proj(backbone(x))
        logits = arc(feats, y)
        loss   = criterion(logits, y)

        opt.zero_grad(); loss.backward(); opt.step()

    # ---- scheduler ----
    (warm if ep <= 2 else sched).step()

    # ---- validation ----
    backbone.eval(); proj.eval(); metric.reset()
    with torch.no_grad():
        # pre-normalize weight matrix once per epoch
        Wn = nn.functional.normalize(arc.W, dim=1)      # [C,512]
        for x, y in val_dl:                             # val_dl from Cell 2
            x, y = x.to(device), y.to(device)
            feats = proj(backbone(x))
            fnorm = nn.functional.normalize(feats, dim=1)
            preds = torch.matmul(fnorm, Wn.T).argmax(1) # cosine scores
            metric.update(preds, y)

    acc = metric.compute().item()
    print(f"Epoch {ep:03d} | Val Macro-Acc {acc:.4f}")

    if acc > best_acc:
        best_acc = acc
        torch.save({'backbone': backbone.state_dict(),
                    'proj'    : proj.state_dict(),
                    'arc'     : arc.state_dict()}, CKPT)
        print("  🔥 new best saved")

print(f"\n▶ Best validation Macro-Accuracy: {best_acc:.4f}")


Epoch 001 | Val Macro-Acc 0.1285
  🔥 new best saved
Epoch 002 | Val Macro-Acc 0.3623
  🔥 new best saved
Epoch 003 | Val Macro-Acc 0.4680
  🔥 new best saved
Epoch 004 | Val Macro-Acc 0.4784
  🔥 new best saved
Epoch 005 | Val Macro-Acc 0.5522
  🔥 new best saved
Epoch 006 | Val Macro-Acc 0.5100
Epoch 007 | Val Macro-Acc 0.5460
Epoch 008 | Val Macro-Acc 0.5932
  🔥 new best saved
Epoch 009 | Val Macro-Acc 0.6386
  🔥 new best saved
Epoch 010 | Val Macro-Acc 0.6559
  🔥 new best saved
Epoch 011 | Val Macro-Acc 0.7068
  🔥 new best saved
Epoch 012 | Val Macro-Acc 0.6752
Epoch 013 | Val Macro-Acc 0.6479
Epoch 014 | Val Macro-Acc 0.5215
Epoch 015 | Val Macro-Acc 0.5513
Epoch 016 | Val Macro-Acc 0.6622
Epoch 017 | Val Macro-Acc 0.6522
Epoch 018 | Val Macro-Acc 0.6744
Epoch 019 | Val Macro-Acc 0.6935
Epoch 020 | Val Macro-Acc 0.7223
  🔥 new best saved
Epoch 021 | Val Macro-Acc 0.7021
Epoch 022 | Val Macro-Acc 0.6859
Epoch 023 | Val Macro-Acc 0.7516
  🔥 new best saved
Epoch 024 | Val Macro-Acc 0.6318

In [23]:
# ================================================================
# Cell 4 : build FIQA-filtered gallery + sweep cosine τ
# ================================================================

# 0️⃣ Install InsightFace & ONNX runtime (only once)
!{sys.executable} -m pip install -q insightface onnxruntime-gpu

import sys, os, gc
import numpy as np
import torch
import torch.nn as nn
import timm
from pathlib import Path
from PIL import Image
from tqdm.auto import tqdm
from torchvision import transforms as T
from insightface.app import FaceAnalysis

# ── device & paths ──────────────────────────────────────────────
device    = 'cuda' if torch.cuda.is_available() else 'cpu'
ROOT      = Path('/kaggle/input/identity-employees-in-surveillance-cctv/dataset')
EXTRA_DIR = Path('/kaggle/tmp_ref_frames')

# ── reload trained backbone + projection ─────────────────────────
CKPT     = '/kaggle/working/effb0_arcface.pt'
backbone = timm.create_model('efficientnet_b0', pretrained=False,
                             num_classes=0, global_pool='avg').to(device)
proj     = nn.Linear(backbone.num_features, 512, bias=False).to(device)

ck = torch.load(CKPT, map_location=device)
backbone.load_state_dict(ck['backbone'])
proj.load_state_dict(    ck['proj'])
backbone.eval(); proj.eval()

# ── FIQA quality model ───────────────────────────────────────────
qa = FaceAnalysis(name='buffalo_s')
qa.prepare(ctx_id=0)

# ── transforms for gallery & val ─────────────────────────────────
val_tfms = T.Compose([
    T.Resize(256), T.CenterCrop(224),
    T.ToTensor(),
    T.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225]),
])

# ── build raw gallery (static JPEGs + video frames) ─────────────
ref_paths, ref_labels, ref_embs = [], [], []

# 1) high-quality reference photos
for emp_dir in (ROOT/'reference_faces').iterdir():
    for jpg in emp_dir.glob('*.jpg'):
        img = Image.open(jpg).convert('RGB')
        ref_paths.append(jpg)
        ref_labels.append(emp_dir.name)
        with torch.no_grad():
            emb = proj(backbone(val_tfms(img).unsqueeze(0).to(device))).squeeze().cpu()
        ref_embs.append(emb)

# 2) extracted video frames
for jpg in EXTRA_DIR.glob('*.jpg'):
    img = Image.open(jpg).convert('RGB')
    ref_paths.append(jpg)
    ref_labels.append(jpg.name.split('_')[0])
    with torch.no_grad():
        emb = proj(backbone(val_tfms(img).unsqueeze(0).to(device))).squeeze().cpu()
    ref_embs.append(emb)

print("Raw gallery size :", len(ref_embs))

# ── FIQA filtering: drop bottom 30% by det_score ────────────────
scores = []
for p in tqdm(ref_paths, desc="FIQA scoring"):
    pil = Image.open(p).convert('RGB')
    arr = np.array(pil)                 # H×W×3
    faces = qa.get(arr)                # returns list of Face objects
    det_score = faces[0].det_score if faces else 0.0
    scores.append(det_score)

threshold = np.percentile(scores, 30)
keep_ix   = [i for i,s in enumerate(scores) if s >= threshold]

ref_embs   = torch.stack([ref_embs[i]    for i in keep_ix])
ref_labels = [ref_labels[i] for i in keep_ix]
ref_norm   = nn.functional.normalize(ref_embs, dim=1)
print("Filtered gallery:", ref_norm.shape)

# ── embed validation CCTV set & sweep τ ─────────────────────────
val_embs, val_true = [], []
idx2emp = {v:k for k,v in emp2idx.items()}

for xb, yb in tqdm(val_dl, desc="val embeddings"):
    xb = xb.to(device)
    with torch.no_grad():
        feats = backbone(xb)           # [B, features]
        embs  = proj(feats).cpu()      # [B, 512]
    val_embs.append(embs)
    val_true.extend([idx2emp[int(i)] for i in yb])

val_embs = torch.cat(val_embs, dim=0)  # [N_val, 512]

def macro_acc(true, pred):
    classes = sorted(set(true))
    return np.mean([
        np.mean([t==p for t,p in zip(true,pred) if t==c])
        for c in classes
    ])

best_tau, best_ma = None, -1
for τ in np.linspace(0.30, 0.55, 26):
    preds = []
    for e in val_embs:
        sims = ref_norm @ nn.functional.normalize(e, dim=0)
        j    = sims.argmax().item()
        preds.append(ref_labels[j] if sims[j] >= τ else "unknown")
    m = macro_acc(val_true, preds)
    print(f"τ={τ:.2f} → Val Macro-Acc {m:.4f}")
    if m > best_ma:
        best_ma, best_tau = m, τ

print(f"\n▶ Best τ = {best_tau:.2f}  (Val Macro-Acc = {best_ma:.4f})")

# ── save gallery + threshold for Cell 5 ─────────────────────────
torch.save({
    'ref_norm':   ref_norm,
    'ref_labels': ref_labels,
    'best_tau':   best_tau
}, '/kaggle/working/gallery.pt')

# cleanup
gc.collect()


Applied providers: ['CUDAExecutionProvider', 'CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}, 'CUDAExecutionProvider': {'sdpa_kernel': '0', 'use_tf32': '1', 'fuse_conv_bias': '0', 'prefer_nhwc': '0', 'tunable_op_max_tuning_duration_ms': '0', 'enable_skip_layer_norm_strict_mode': '0', 'tunable_op_tuning_enable': '0', 'tunable_op_enable': '0', 'use_ep_level_unified_stream': '0', 'device_id': '0', 'has_user_compute_stream': '0', 'gpu_external_empty_cache': '0', 'cudnn_conv_algo_search': 'EXHAUSTIVE', 'cudnn_conv1d_pad_to_nc1d': '0', 'gpu_mem_limit': '18446744073709551615', 'gpu_external_alloc': '0', 'gpu_external_free': '0', 'arena_extend_strategy': 'kNextPowerOfTwo', 'do_copy_in_default_stream': '1', 'enable_cuda_graph': '0', 'user_compute_stream': '0', 'cudnn_conv_use_max_workspace': '1'}}
find model: /root/.insightface/models/buffalo_s/1k3d68.onnx landmark_3d_68 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CUDAExecutionProvider', 'CPUExecutionProvider'], with o

FIQA scoring:   0%|          | 0/21867 [00:00<?, ?it/s]

Filtered gallery: torch.Size([21867, 512])


val embeddings:   0%|          | 0/4 [00:00<?, ?it/s]

τ=0.30 → Val Macro-Acc 0.7750
τ=0.31 → Val Macro-Acc 0.7750
τ=0.32 → Val Macro-Acc 0.7750
τ=0.33 → Val Macro-Acc 0.7750
τ=0.34 → Val Macro-Acc 0.7750
τ=0.35 → Val Macro-Acc 0.7750
τ=0.36 → Val Macro-Acc 0.7750
τ=0.37 → Val Macro-Acc 0.7750
τ=0.38 → Val Macro-Acc 0.7750
τ=0.39 → Val Macro-Acc 0.7750
τ=0.40 → Val Macro-Acc 0.7750
τ=0.41 → Val Macro-Acc 0.7750
τ=0.42 → Val Macro-Acc 0.7750
τ=0.43 → Val Macro-Acc 0.7750
τ=0.44 → Val Macro-Acc 0.7750
τ=0.45 → Val Macro-Acc 0.7750
τ=0.46 → Val Macro-Acc 0.7750
τ=0.47 → Val Macro-Acc 0.7750
τ=0.48 → Val Macro-Acc 0.7750
τ=0.49 → Val Macro-Acc 0.7750
τ=0.50 → Val Macro-Acc 0.7750
τ=0.51 → Val Macro-Acc 0.7750
τ=0.52 → Val Macro-Acc 0.7750
τ=0.53 → Val Macro-Acc 0.7750
τ=0.54 → Val Macro-Acc 0.7750
τ=0.55 → Val Macro-Acc 0.7750

▶ Best τ = 0.30  (Val Macro-Acc = 0.7750)


100

In [26]:
# ================================================================
# Cell 4B : grid-search (P_TH, τ) on validation set
# ================================================================

import numpy as np
import torch
import torch.nn as nn
import timm
from itertools import product

device      = 'cuda' if torch.cuda.is_available() else 'cpu'
NUM_CLASSES = len(employee_ids)

# ── 1️⃣ Redefine ArcFace to match your checkpoint keys ────────────
class ArcFace(nn.Module):
    def __init__(self, in_f, out_f, s=30.0, m=0.50):
        super().__init__()
        self.W = nn.Parameter(torch.randn(out_f, in_f))
        nn.init.xavier_uniform_(self.W)
        self.s, self.m = s, m
        self.cos_m, self.sin_m = np.cos(m), np.sin(m)
        self.th, self.mm = np.cos(np.pi - m), np.sin(np.pi - m) * m

    def forward(self, x, y):
        x_n = nn.functional.normalize(x)
        w_n = nn.functional.normalize(self.W)
        cos = nn.functional.linear(x_n, w_n)
        sin = torch.sqrt(1.0 - torch.clamp(cos**2, 0, 1))
        phi = torch.where(cos > self.th,
                          cos * self.cos_m - sin * self.sin_m,
                          cos - self.mm)
        one_hot = torch.zeros_like(cos, device=x.device)
        one_hot.scatter_(1, y.view(-1,1), 1.0)
        logits = (one_hot * phi + (1.0 - one_hot) * cos) * self.s
        return logits

# ── 2️⃣ Reload your classifier head to get soft-max predictions ───
ck = torch.load('/kaggle/working/effb0_arcface.pt', map_location=device)

backbone = timm.create_model('efficientnet_b0', pretrained=False,
                             num_classes=0, global_pool='avg').to(device)
proj     = nn.Linear(backbone.num_features, 512, bias=False).to(device)
arc      = ArcFace(512, NUM_CLASSES).to(device)

backbone.load_state_dict(ck['backbone'])
proj.load_state_dict(ck['proj'])
arc.load_state_dict( ck['arc'] )

backbone.eval(); proj.eval(); arc.eval()

@torch.no_grad()
def softmax_vec(x):
    feats  = proj(backbone(x))
    dummy  = torch.zeros(len(x), dtype=torch.long, device=device)
    logits = arc(feats, dummy)
    return torch.softmax(logits, dim=1)

# ── 3️⃣ Precompute soft-max on your val set ───────────────────────
pvecs = []
for imgs, _ in val_dl:
    pvecs.append( softmax_vec(imgs.to(device)).cpu() )
pvecs_val = torch.cat(pvecs)  # [N_val, NUM_CLASSES]

# ── 4️⃣ Macro-accuracy helper ───────────────────────────────────
def macro_acc(true, pred):
    classes = sorted(set(true))
    return np.mean([
        np.mean([t==p for t,p in zip(true,pred) if t==c])
        for c in classes
    ])

# (we assume the following variables are in scope from Cell 4):
#   ref_norm    # torch.Tensor [N_ref, 512]
#   ref_labels  # list[str] of length N_ref
#   best_tau    # float from Cell 4
#   val_embs    # torch.Tensor [N_val, 512]
#   val_true    # list[str] of length N_val

# ── 5️⃣ Grid-search over P_TH ∈ [0.50,0.70] and τ ±0.03 ───────────
P_grid = np.arange(0.50, 0.71, 0.02)
T_grid = np.arange(best_tau-0.03, best_tau+0.031, 0.01)

best_pair, best_ma = None, -1
print(f"Searching P×τ grid with {len(P_grid)}×{len(T_grid)} combos…")

for P_TH, τ in product(P_grid, T_grid):
    preds = []
    for prob_vec, emb in zip(pvecs_val, val_embs):
        top_p, top_i = prob_vec.max(0)
        if top_p >= P_TH:
            preds.append(employee_ids[int(top_i)])
        else:
            sims = ref_norm @ nn.functional.normalize(emb, dim=0)
            j    = int(sims.argmax())
            preds.append(ref_labels[j] if sims[j] >= τ else "unknown")
    m = macro_acc(val_true, preds)
    if m > best_ma:
        best_ma, best_pair = m, (P_TH, τ)
    print(f"P={P_TH:.2f}, τ={τ:.2f} → Val Macro-Acc {m:.4f}")

print(f"\n▶ BEST thresholds: P_TH = {best_pair[0]:.2f}, τ = {best_pair[1]:.2f}   (Val Macro-Acc = {best_ma:.4f})")


Searching P×τ grid with 11×7 combos…
P=0.50, τ=0.27 → Val Macro-Acc 0.7988
P=0.50, τ=0.28 → Val Macro-Acc 0.7988
P=0.50, τ=0.29 → Val Macro-Acc 0.7988
P=0.50, τ=0.30 → Val Macro-Acc 0.7988
P=0.50, τ=0.31 → Val Macro-Acc 0.7988
P=0.50, τ=0.32 → Val Macro-Acc 0.7988
P=0.50, τ=0.33 → Val Macro-Acc 0.7988
P=0.52, τ=0.27 → Val Macro-Acc 0.7988
P=0.52, τ=0.28 → Val Macro-Acc 0.7988
P=0.52, τ=0.29 → Val Macro-Acc 0.7988
P=0.52, τ=0.30 → Val Macro-Acc 0.7988
P=0.52, τ=0.31 → Val Macro-Acc 0.7988
P=0.52, τ=0.32 → Val Macro-Acc 0.7988
P=0.52, τ=0.33 → Val Macro-Acc 0.7988
P=0.54, τ=0.27 → Val Macro-Acc 0.8037
P=0.54, τ=0.28 → Val Macro-Acc 0.8037
P=0.54, τ=0.29 → Val Macro-Acc 0.8037
P=0.54, τ=0.30 → Val Macro-Acc 0.8037
P=0.54, τ=0.31 → Val Macro-Acc 0.8037
P=0.54, τ=0.32 → Val Macro-Acc 0.8037
P=0.54, τ=0.33 → Val Macro-Acc 0.8037
P=0.56, τ=0.27 → Val Macro-Acc 0.8037
P=0.56, τ=0.28 → Val Macro-Acc 0.8037
P=0.56, τ=0.29 → Val Macro-Acc 0.8037
P=0.56, τ=0.30 → Val Macro-Acc 0.8037
P=0.56, τ=0.3

In [ ]:
# Pk={}
# for cls_idx, emp in enumerate(employee_ids):
#     conf=[]
#     with torch.no_grad():
#         for imgs, labels in val_dl:
#             probs = softmax_vec(imgs.to(device)).cpu()
#             mask  = labels==cls_idx
#             conf.extend(probs[mask, cls_idx].numpy())
#     Pk[emp] = np.percentile(conf, 10)   # 10-th percentile

# print("Example thresholds:", list(Pk.items())[:4])


In [27]:
# ================================================================
# Cell 5 : hybrid soft-max + gallery → submission.csv
# ================================================================
import torch
import pandas as pd
import timm
from PIL import Image

device      = 'cuda' if torch.cuda.is_available() else 'cpu'
NUM_CLASSES = len(employee_ids)
CKPT        = '/kaggle/working/effb0_arcface.pt'

# ── 1️⃣ Plug in your best thresholds ────────────────────────────
P_TH = 0.54   # from Cell 4B
C_TH = 0.27   # from Cell 4B

# ── 2️⃣ Reload backbone + ArcFace head ─────────────────────────
# (ArcFace class must already be defined in your notebook)
backbone = timm.create_model('efficientnet_b0', pretrained=False,
                             num_classes=0, global_pool='avg').to(device)
proj     = torch.nn.Linear(backbone.num_features, 512, bias=False).to(device)
arc      = ArcFace(512, NUM_CLASSES).to(device)

ck = torch.load(CKPT, map_location=device)
backbone.load_state_dict( ck['backbone'] )
proj.load_state_dict(     ck['proj']     )
arc.load_state_dict(      ck['arc']      )

backbone.eval(); proj.eval(); arc.eval()

# ── 3️⃣ Define helper functions ────────────────────────────────
@torch.no_grad()
def softmax_vec(x):
    feats  = proj(backbone(x))
    dummy  = torch.zeros(len(x), dtype=torch.long, device=device)
    logits = arc(feats, dummy)
    return torch.softmax(logits, dim=1)

@torch.no_grad()
def embed(timg):
    t = val_tfms(timg).unsqueeze(0).to(device)
    return proj(backbone(t)).squeeze().cpu()

# ── 4️⃣ Inference loop ─────────────────────────────────────────
rows = []
for img_path in sorted(TEST_IMG_DIR.glob('*.jpg')):
    pil = Image.open(img_path).convert('RGB')
    # classifier branch with softmax
    x      = val_tfms(pil).unsqueeze(0).to(device)
    pvec   = softmax_vec(x)[0].cpu()
    top_p, top_i = pvec.max(0)

    # gallery branch with cosine similarity
    emb    = embed(pil)
    emb_n  = torch.nn.functional.normalize(emb, dim=0)
    sims   = ref_norm @ emb_n         # ref_norm & ref_labels from Cell 4
    j      = int(sims.argmax())

    # hybrid decision
    if top_p >= P_TH:
        pred = employee_ids[int(top_i)]
    elif sims[j] >= C_TH:
        pred = ref_labels[j]
    else:
        pred = "unknown"

    rows.append((img_path.name, pred))

# ── 5️⃣ Write submission.csv ───────────────────────────────────
sub = pd.DataFrame(rows, columns=['image_name', 'employee_id'])
sub.to_csv('/kaggle/working/submission.csv', index=False)
print("✅ submission.csv saved:", sub.shape)
display(sub.head())


✅ submission.csv saved: (636, 2)


,image_name,employee_id
0,face_0002.jpg,emp019
1,face_0005.jpg,emp014
2,face_0009.jpg,emp016
3,face_0012.jpg,emp016
4,face_0014.jpg,emp002
